In [2]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import shutil
torch.__version__

'1.10.0+cu111'

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

device(type='cuda')

In [5]:
df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/page_train.csv')
df_test= pd.read_csv('/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/page_test.csv')

In [6]:
targetList= ['vector spaces',
 'pos_def_matrices',
 'eigenvec_val',
 'determinants',
 'orthogonality',
 'linear_prog',
 'gauss elim',
 'computations']

In [7]:
df_test = pd.concat([df_test,pd.get_dummies(df_test['label'])], axis=1)
df_test.drop(columns='label')
print(df_test.shape)
df_test.head()

(90, 10)


,text,label,computations,determinants,eigenvec_val,gauss elim,linear_prog,orthogonality,pos_def_matrices,vector spaces
0,Chapter 2 Vector Spacesand only a few hundred ...,vector spaces,0,0,0,0,0,0,0,1
1,Chapter 5 Eigenvalues and EigenvectorsProblems...,eigenvec_val,0,0,1,0,0,0,0,0
2,of the columns of Q = I:That is not the only o...,orthogonality,0,0,0,0,0,1,0,0
3,Chapter 2 Vector SpacesYou must notice that th...,vector spaces,0,0,0,0,0,0,0,1
4,5.2 Diagonalization of a MatrixIf A is inverti...,eigenvec_val,0,0,1,0,0,0,0,0


In [8]:
df_train = pd.concat([df_train,pd.get_dummies(df_train['label'])], axis=1)
df_train = df_train.drop(columns='label')
print(df_train.shape)
df_train.head()

(358, 9)


,text,computations,determinants,eigenvec_val,gauss elim,linear_prog,orthogonality,pos_def_matrices,vector spaces
0,1.4 Matrix Notation and Matrix Multiplication1...,0,0,0,1,0,0,0,0
1,5.4 Differential Equations and eAt−ω2 = λ. Eve...,0,0,1,0,0,0,0,0
2,Chapter 5 Eigenvalues and EigenvectorsAt the l...,0,0,1,0,0,0,0,0
3,5.2 Diagonalization of a MatrixIf eigenvectors...,0,0,1,0,0,0,0,0
4,Which previous problem justiﬁes the middle ste...,0,0,0,0,0,1,0,0


In [ ]:
pip install transformers

In [10]:
from transformers import BertTokenizer, BertModel

In [11]:
tokenizer= BertTokenizer.from_pretrained("bert-base-uncased")

In [12]:
df_train1=df_train.sample(frac=0.5, random_state= 200).reset_index(drop=True)
page_df= df_train.drop(df_train1.index).reset_index(drop=True)
df_train= df_train1
page_df.head(), df_train.head()

(                                                text  ...  vector spaces
 0  5.3 Difference Equations and Powers AkExample ...  ...              0
 1  6.3 Singular Value DecompositionOf course Σ co...  ...              0
 2  Chapter 6 Positive Deﬁnite Matricesinstead of ...  ...              0
 3  Chapter 5 Eigenvalues and EigenvectorsThe only...  ...              0
 4  Chapter 2 Vector SpacesMost sets of four vecto...  ...              1
 
 [5 rows x 9 columns],
                                                 text  ...  vector spaces
 0  1.5 Triangular Factors and Row Exchanges31. Tr...  ...              0
 1  2.2 Solving Ax = 0 and Ax = bThere must be inﬁ...  ...              1
 2  Chapter 1 Matrices and Gaussian Elimination25....  ...              0
 3  Chapter 7 Computations with Matricesx0 = 0, r0...  ...              0
 4  3.1 Orthogonal Vectors and Subspacesequations ...  ...              0
 
 [5 rows x 9 columns])

In [13]:
df_train1.shape, df_train.shape, page_df.shape 

((179, 9), (179, 9), (179, 9))

In [14]:
MAX_LEN= 512
BATCH_SIZE= 14
EPOCHS= 10
LR= 1e-4

In [15]:
class myDataset(torch.utils.data.Dataset):
    def __init__(self,df, tokenizer, max_len):
        self.df = df
        self.tokenizer= tokenizer
        self.max_len = max_len
        self.text= df['text']
        self.targets = self.df[targetList].values
    def __len__(self):
        return len(self.text)
    def __getitem__(self,index):
        
        text= str(self.text[index])
        text=  " ".join(text.split())
        
        inputs= self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids= True,
            truncation=True,
            return_attention_mask=True,
            return_tensors= 'pt'
        )
        
        return {
            'input_ids':inputs['input_ids'].flatten(),
            'attention_mask':inputs['attention_mask'].flatten(),
            'token_type_ids':inputs['token_type_ids'].flatten(),
            'targets':torch.FloatTensor(self.targets[index])
        }


In [16]:
train_df= df_train.sample(frac=0.8,random_state=200).reset_index(drop=True)
val_df= df_train.drop(train_df.index).reset_index(drop=True)
train_df.shape, val_df.shape

((143, 9), (36, 9))

In [17]:
train_dataset=myDataset(train_df,tokenizer, MAX_LEN)
val_dataset = myDataset(val_df,tokenizer,MAX_LEN)

In [18]:
trainLoader = torch.utils.data.DataLoader(
    train_dataset,
    shuffle=True,
    batch_size = BATCH_SIZE,
    num_workers= 0 
    )
valLoader = torch.utils.data.DataLoader(
    val_dataset,
    shuffle=False,
    batch_size = BATCH_SIZE,
    num_workers= 0 
    )

In [19]:
def load_ckp(ckpt_path,model,optimizer):
    checkpoint= torch.load(ckpt_path)
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    valid_loss_min= checkpoint['valid_loss_min']
    return model, optimizer, checkpoint['epoch'],valid_loss_min.item()
def save_ckp(state, is_best, ckpt_path,best_model_path):
    torch.save(state, ckpt_path)
    if is_best:
        best_path = best_model_path
        shutil.copyfile(ckpt_path,best_path)

In [ ]:
class BERTModel(nn.Module):
    def __init__(self):
        super(BERTModel,self).__init__()
        self.bert= BertModel.from_pretrained("bert-base-uncased", return_dict= True)
        self.dropout = nn.Dropout(0.3) 
        self.linear =nn.Linear(768,8)
    def forward(self, input_ids,attention_mask,token_type_ids):
        output = self.bert(input_ids,attention_mask,token_type_ids)
        outputDropout= self.dropout(output.pooler_output)
        output= self.linear(outputDropout)
        return output
model = BERTModel()
model.to(device)

In [21]:
def loss_fn(outputs, targets):
    return nn.BCEWithLogitsLoss()(outputs,targets)
def accuracy(output,targets):
  final_output = torch.sigmoid(output).cpu().detach().numpy()
  return np.sum( np.argmax(targets.cpu().numpy(),axis=1) == np.argmax(final_output, axis=1) ) / BATCH_SIZE *100

optimizer= torch.optim.Adam(params=model.parameters(),lr=LR)

In [22]:
train_lossList=[]
train_accList=[]
val_lossList=[]
val_accList=[]

In [23]:
def train_model(epochs, trainLoader,valLoader,model,optimizer,ckpt_path,best_model_path):
    val_loss_min= np.Inf
    for epoch in range(1, epochs+1):
        print("epoch[{}]".format(epoch))
        train_loss= 0
        val_loss= 0
        for index, batch in enumerate(trainLoader):
            input_ids= batch['input_ids'].to(device, dtype= torch.long)
            attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
            token_type_ids= batch['token_type_ids'].to(device, dtype= torch.long)
            targets = batch['targets'].to(device, dtype= torch.float)

            output= model(input_ids, attention_mask, token_type_ids)
            
            optimizer.zero_grad()
            loss= loss_fn(output,targets)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_acc= accuracy(output,targets)
            train_loss=train_loss + ((1/(index+1))*(loss.item()-train_loss))
            print("TRAIN BATCH {}/{}====>train loss[{:.8f}] | train accuracy[{:.8f}%]".format(index+1,len(trainLoader) , train_loss, train_acc ))
            #train_lossList+=[train_loss]
            #train_accList+=[train_acc]
        model.eval() 
        with torch.no_grad():
            for index, batch in enumerate(valLoader):
                print("validation batch index:{}/{}".format(index,len(valLoader)))
                input_ids= batch['input_ids'].to(device, dtype= torch.long)
                attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
                token_type_ids= batch['token_type_ids'].to(device, dtype= torch.long)
                targets = batch['targets'].to(device, dtype= torch.float)
                
                output =model(input_ids,attention_mask,token_type_ids)
                val_acc= accuracy(output,targets)
                loss= loss_fn(output,targets)
                val_loss= (val_loss + (1/(index+1))*(loss.item()-val_loss))
                print("VALIDATION BATCH {}/{}====>val loss[{:.8f}] | val accuracy[{:.8f}]".format(index+1,len(valLoader) , val_loss, val_acc ))
                #val_lossList+=[val_loss]
                #val_accList+=[val_acc]
        checkpoint ={
            'epoch': epoch+1,
            'valid_loss_min':val_loss,
            'state_dict':model.state_dict(),
            'optimizer':optimizer.state_dict()
        }
        save_ckp(checkpoint,False,ckpt_path,best_model_path)

        if val_loss < val_loss_min:
          print("previous Val_loss_min={:.8f}; new val_loss_min={:.8f}".format(val_loss_min, val_loss))
          save_ckp(checkpoint, True, ckpt_path, best_model_path)
          print("SAVED")
          val_loss_min = val_loss
        print("epoch {} end".format(epoch))
    return model

In [24]:
ckpt_path = "/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/curr_path"
best_model_path = "/content/drive/MyDrive/Colab Notebooks/proj/Data/pages/best.pt"

In [25]:
torch.cuda.empty_cache()
trained_model=train_model(EPOCHS,trainLoader,valLoader,model, optimizer,ckpt_path,best_model_path)

epoch[1]
TRAIN BATCH 1/11====>train loss[0.66226238] | train accuracy[0.00000000%]
TRAIN BATCH 2/11====>train loss[0.65520260] | train accuracy[21.42857143%]
TRAIN BATCH 3/11====>train loss[0.63531009] | train accuracy[7.14285714%]
TRAIN BATCH 4/11====>train loss[0.61307171] | train accuracy[7.14285714%]
TRAIN BATCH 5/11====>train loss[0.59028550] | train accuracy[0.00000000%]
TRAIN BATCH 6/11====>train loss[0.56704230] | train accuracy[21.42857143%]
TRAIN BATCH 7/11====>train loss[0.54775502] | train accuracy[0.00000000%]
TRAIN BATCH 8/11====>train loss[0.53160523] | train accuracy[0.00000000%]
TRAIN BATCH 9/11====>train loss[0.51767208] | train accuracy[7.14285714%]
TRAIN BATCH 10/11====>train loss[0.50426894] | train accuracy[28.57142857%]
TRAIN BATCH 11/11====>train loss[0.49428939] | train accuracy[7.14285714%]
validation batch index:0/3
VALIDATION BATCH 1/3====>val loss[0.36911926] | val accuracy[42.85714286]
validation batch index:1/3
VALIDATION BATCH 2/3====>val loss[0.36848947

In [26]:
train_df2= page_df.sample(frac=0.8,random_state=200).reset_index(drop=True)

In [28]:
val_df2= page_df.drop(train_df2.index).reset_index(drop=True)

In [29]:
train_dataset2=myDataset(train_df2,tokenizer, MAX_LEN)
val_dataset2 = myDataset(val_df2,tokenizer,MAX_LEN)

In [30]:
trainLoader2 = torch.utils.data.DataLoader(
    train_dataset2,
    shuffle=True,
    batch_size = BATCH_SIZE,
    num_workers= 0 
    )
valLoader2 = torch.utils.data.DataLoader(
    val_dataset2,
    shuffle=False,
    batch_size = BATCH_SIZE,
    num_workers= 0 
    )

In [31]:
trained_model=train_model(EPOCHS,trainLoader2,valLoader2,model, optimizer,ckpt_path,best_model_path)

epoch[1]
TRAIN BATCH 1/11====>train loss[0.06591234] | train accuracy[100.00000000%]
TRAIN BATCH 2/11====>train loss[0.09367062] | train accuracy[85.71428571%]
TRAIN BATCH 3/11====>train loss[0.10177842] | train accuracy[85.71428571%]
TRAIN BATCH 4/11====>train loss[0.09901355] | train accuracy[85.71428571%]
TRAIN BATCH 5/11====>train loss[0.09352551] | train accuracy[100.00000000%]
TRAIN BATCH 6/11====>train loss[0.08731181] | train accuracy[100.00000000%]
TRAIN BATCH 7/11====>train loss[0.09494212] | train accuracy[85.71428571%]
TRAIN BATCH 8/11====>train loss[0.09714813] | train accuracy[85.71428571%]
TRAIN BATCH 9/11====>train loss[0.09633655] | train accuracy[92.85714286%]
TRAIN BATCH 10/11====>train loss[0.09365807] | train accuracy[92.85714286%]
TRAIN BATCH 11/11====>train loss[0.09120258] | train accuracy[21.42857143%]
validation batch index:0/3
VALIDATION BATCH 1/3====>val loss[0.10591528] | val accuracy[92.85714286]
validation batch index:1/3
VALIDATION BATCH 2/3====>val loss

In [32]:
test_dataset = myDataset(df_test,tokenizer,MAX_LEN)
testLoader = torch.utils.data.DataLoader(
    test_dataset,
    shuffle=False,
    batch_size = BATCH_SIZE,
    num_workers= 0 
    )
df_test.shape

(90, 10)

In [37]:
#testing
test_accList =[]
test_loss=0
model.eval() 
with torch.no_grad():
  for index, batch in enumerate(testLoader):
    input_ids= batch['input_ids'].to(device, dtype= torch.long)
    attention_mask = batch['attention_mask'].to(device, dtype= torch.long)
    token_type_ids= batch['token_type_ids'].to(device, dtype= torch.long)
    targets = batch['targets'].to(device, dtype= torch.float)
    
    output =model(input_ids,attention_mask,token_type_ids)
    
    loss= loss_fn(output,targets)
    test_acc= accuracy(output,targets)
    test_accList+= [test_acc]
    test_loss = test_loss + ((1 / (index + 1)) * (loss.item() - test_loss))
    print("TEST BATCH {}/{}====>test loss[{:.8f}] | test accuracy[{:.8f}]".format(index+1,len(testLoader) , test_loss, test_acc ))
    #final_output = torch.sigmoid(output).cpu().detach().numpy()
    #print(np.argmax(targets.cpu().numpy(),axis=1), np.argmax(final_output, axis=1) )
print("AVG Accuracy = ", sum(test_accList)/len(test_accList))

TEST BATCH 1/7====>test loss[0.10268328] | test accuracy[85.71428571]
TEST BATCH 2/7====>test loss[0.06064245] | test accuracy[100.00000000]
TEST BATCH 3/7====>test loss[0.05761292] | test accuracy[92.85714286]
TEST BATCH 4/7====>test loss[0.07725312] | test accuracy[85.71428571]
TEST BATCH 5/7====>test loss[0.07983852] | test accuracy[85.71428571]
TEST BATCH 6/7====>test loss[0.07501613] | test accuracy[92.85714286]
TEST BATCH 7/7====>test loss[0.06702940] | test accuracy[42.85714286]
AVG Accuracy =  83.6734693877551
